In [1]:
import cvxpy as cp
import numpy as np
import random
import math
import time
from collections import deque

In [2]:
print(cp.installed_solvers())

['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'MOSEK', 'OSQP', 'SCIPY', 'SCS']


In [ ]:
def Kendall_Tau_Dist(first, second):
    mappedrank = []
    for i in range(len(second)):
        mappedrank.append(first.index(second[i]))
    cost, blank = mergesort(mappedrank)
    return cost

#mergesort method for nlogn kendall tau distance
def mergesort(ranking):
    if len(ranking) <= 1:
        return 0, ranking
    leftsum, leftrank = mergesort(ranking[:len(ranking)//2])
    rightsum, rightrank = mergesort(ranking[len(ranking)//2:])
    csum = leftsum + rightsum
    leftindex = 0
    rightindex = 0
    outrank = []
    while leftindex < len(leftrank) and rightindex < len(rightrank):
        if leftrank[leftindex] < rightrank[rightindex]:
            outrank.append(leftrank[leftindex])
            leftindex += 1
        else:
            outrank.append(rightrank[rightindex])
            csum += len(leftrank) - leftindex
            rightindex += 1
    if leftindex < len(leftrank):
        outrank += leftrank[leftindex:]
    if rightindex < len(rightrank):
        outrank += rightrank[rightindex:]
    return csum, outrank
    
def Get_Objective_Value(query, rankings):
    median_cost = 0
    for rank in rankings:
        median_cost += Kendall_Tau_Dist(query, rank)
    return median_cost

#helper function to return the weighted tournament corresponding to the rank aggregation problem
def Get_Frac_Tournament(rankings):
    element_count = len(rankings[0])
    frac_tournament = np.ndarray((element_count, element_count))
    for i in range(element_count):
        for j in range(element_count):
            frac_tournament[i][j] = 0

    for ranking in rankings:
        for i in range(len(ranking)):
            for j in range(i+1, len(ranking)):
                frac_tournament[ranking[i]][ranking[j]] += 1
    for i in range(element_count):
        for j in range(element_count):
            frac_tournament[i][j] = frac_tournament[i][j] / len(rankings)

    return frac_tournament

#helper function to recover ordering from acyclic tournament
def Topological_Sort(adj):
    n = len(adj)  
    in_degree = [0] * n

    for i in range(n):
        for j in range(n):
            if adj[i][j] > 0.5:
                in_degree[j] += 1

    queue = deque()
    for i in range(n):
        if in_degree[i] == 0:
            queue.append(i)

    topo_sort = []

    while queue:
        node = queue.popleft()
        topo_sort.append(node)

        for j in range(n):
            if adj[node][j] > 0.5:
                in_degree[j] -= 1
                if in_degree[j] == 0:
                    queue.append(j)

    return topo_sort

In [ ]:
#Input: set of rankings
#Returns: the optimal median ranking by using an ILP
#See "Improved Bounds for Computing Kemeny Rankings" for related information.
def NormalILP(rankings):
    element_count = len(rankings[0])
    
    frac_tournament = Get_Frac_Tournament(rankings)
    
    X = cp.Variable(element_count * element_count, boolean = True)
    constraints = []
    
    #constraints that for every pair, one is before the other
    for i in range(element_count):
        for j in range(element_count):
            coeff = np.zeros(element_count * element_count)
            coeff[i*element_count + j] = 1
            coeff[j*element_count + i] = 1
            constraints += [coeff @ X == 1]
    
    #triangle inequality constraint
    #x_ab + x_bc + x_ca >= 1 for any a, b, c
    for i in range(element_count):
        for j in range(element_count):
            if i == j:
                continue
            for k in range(element_count):
                if i == k or j == k:
                    continue
                coeff = np.zeros(element_count * element_count)
                coeff[i*element_count + j] = 1
                coeff[j*element_count + k] = 1
                coeff[k*element_count + i] = 1
                constraints += [coeff @ X >= 1]
    
    edge_weight_coeff = np.empty(element_count * element_count)
    for i in range(element_count):
        for j in range(element_count):
            edge_weight_coeff[i * element_count + j] = frac_tournament[i][j]
    
    problem = cp.Problem(cp.Minimize(edge_weight_coeff @ X), constraints)
    
    print("Constraints done. Solving...")
    problem.solve(solver = cp.SCIP)

    result = X.value.reshape(element_count, -1)
    for i in range(element_count):
        result[i][i] = 0
    result_tp = [[0]*element_count for i in range(element_count)]
    for i in range(element_count):
        for j in range(element_count):
            if i != j:
                result_tp[i][j] = result[j][i]
    
    topo_sorted = Topological_Sort(result_tp)
    print("Objective value: ", Get_Objective_Value(topo_sorted, rankings))
    return topo_sorted

In [ ]:
#Streaming
def greedy_kcenter(x_array):
    #Randomly select a point and add to centers
    randomPointIndex = np.random.randint(0,len(x_array)+1)
    center = x_array[randomPointIndex]

    #Initialize all distances initially to s_1
    point_distances = [Kendall_Tau_Dist(x_array[i], center) for i in range(len(x_array))]

    while len(self.centers) < 1:
        #Get the farthest point to add to centers
        max_point_index = point_distances.index(max(point_distances))
        maximum_dist_point = self.x_array[max_point_index]

        self.centers = np.vstack([self.centers, maximum_dist_point])

        #Update point distances
        point_distances = [min(hf.dist(self.x_array[i], maximum_dist_point), point_distances[i]) for i in range(len(self.x_array))]


    #Get the cost, R
    self.R_val = max(point_distances)
    print("Cost (max) of k-center clustering={:.3f}".format(self.R_val))

    return 
